## Regression Model with TF DenseFeatures

In [3]:
### Dependancies

In [1]:
#this should move to a create lifecycle config
#import libraries
import boto3,sagemaker
import   re, sys, math, json, os, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
# from sagemaker.predictor import csv_serializer   

#this should move to a create lifecycle config
# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/ehrsample2'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")


Success - the MySageMakerInstance is in the us-west-2 region. You will use the 433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [2]:
#this should move to a create lifecycle config
#create S3 Bucket
bucket_name = 'ehrsample2' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [3]:
column_list = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num_label']

In [4]:
#this should move to a start lifecycle config (or should this be a processing job?)

#swiss data set
swiss_dataset_path = "s3://ehrsample2/processedswitzerland.txt"

#old code
#try:
#  urllib.request.urlretrieve ("s3://ehrsample2/processedswitzerland.csv", "processedswitzerland.csv.csv")
#  print('Success: downloaded swiss_setn.csv.')
#except Exception as e:
#  print('Data load error: ',e)

try:
  swiss_df = pd.read_csv(swiss_dataset_path,  names=column_list).replace('?', np.nan)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

#cleveland data set
cleveland_dataset_path = "s3://ehrsample2/processedcleveland.txt"

#old code
#try:
#  urllib.request.urlretrieve ("s3://ehrsample2/processedcleveland.csv", "processedcleveland.csv")
#  print('Success: downloaded cleveland_setn.csv.')
#except Exception as e:
#  print('Data load error: ',e)

try:
  cleveland_df = pd.read_csv(cleveland_dataset_path,  names=column_list).replace('?', np.nan)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.
Success: Data loaded into dataframe.


In [5]:
swissdfsample = swiss_df.head(4)
print(swissdfsample)

   age  sex  cp trestbps  chol  fbs restecg thalach exang oldpeak slope   ca  \
0   32    1   1       95     0  NaN       0     127     0      .7     1  NaN   
1   34    1   4      115     0  NaN     NaN     154     0      .2     1  NaN   
2   35    1   4      NaN     0  NaN       0     130     1     NaN   NaN  NaN   
3   36    1   4      110     0  NaN       0     125     1       1     2  NaN   

  thal  num_label  
0  NaN          1  
1  NaN          1  
2    7          3  
3    6          1  


In [6]:
#old code from pre-S3, can remove
#from https://archive.ics.uci.edu/ml/datasets/Heart+Disease
#swiss_dataset_path = "./data/heart_disease_data/processed_swiss.csv"
#swiss_df = pd.read_csv(swiss_dataset_path).replace('?', np.nan)

In [7]:
#old code from pre-S3, can remove
#cleveland_df = pd.read_csv("./data/heart_disease_data/processed.cleveland.txt",  names=column_list).replace('?', np.nan)

In [8]:
#combine both sources into one
combined_heart_df = pd.concat([swiss_df, cleveland_df])

In [9]:
len(combined_heart_df)

426

In [10]:
dfsample = combined_heart_df.head(4)
print(dfsample)

    age  sex   cp trestbps  chol  fbs restecg thalach exang oldpeak slope  \
0  32.0  1.0  1.0       95   0.0  NaN       0     127     0      .7     1   
1  34.0  1.0  4.0      115   0.0  NaN     NaN     154     0      .2     1   
2  35.0  1.0  4.0      NaN   0.0  NaN       0     130     1     NaN   NaN   
3  36.0  1.0  4.0      110   0.0  NaN       0     125     1       1     2   

    ca thal  num_label  
0  NaN  NaN          1  
1  NaN  NaN          1  
2  NaN    7          3  
3  NaN    6          1  


### Model Preparation
- This simple verson of the dataset has only a three features. 

In [11]:
selected_features = ['sex',  'age', 'trestbps', 'thalach' ]
bp_df = combined_heart_df[selected_features].replace({1:"male", 0:"female"})

In [12]:
# TEST how many rows with at least a single null value
sum(bp_df.apply(lambda x: sum(x.isnull().values), axis = 1)>0)

2

In [13]:
len(bp_df[bp_df['trestbps'].isnull()==True])

2

In [14]:
bp_df.head()

,sex,age,trestbps,thalach
0,male,32.0,95,127
1,male,34.0,115,154
2,male,35.0,NaN,130
3,male,36.0,110,125
4,female,38.0,105,166


In [15]:
#for simplicity will drop rows with null since predictor is null 
clean_df = bp_df.dropna()

In [16]:
clean_df.head()

,sex,age,trestbps,thalach
0,male,32.0,95,127
1,male,34.0,115,154
3,male,36.0,110,125
4,female,38.0,105,166
5,female,38.0,110,156


In [19]:
clean_df['trestbps'].loc[:] = clean_df['trestbps'].loc[:].astype(float)
clean_df['thalach'].loc[:] = clean_df['thalach'].loc[:].astype(float)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [20]:
#adapted from https://www.tensorflow.org/tutorials/structured_data/feature_columns
def df_to_dataset(df, predictor,  batch_size=32):
    df = df.copy()
    labels = df.pop(predictor)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

In [21]:
#split 80 20 train test split - not ideal 
train_dataset = clean_df.sample(frac=0.8,random_state=0)
test_dataset = clean_df.drop(train_dataset.index)

In [22]:
PREDICTOR_FIELD = 'trestbps'
batch_size = 128
train_ds = df_to_dataset(train_dataset, PREDICTOR_FIELD, batch_size=batch_size)
test_ds = df_to_dataset(test_dataset, PREDICTOR_FIELD, batch_size=batch_size)

In [23]:
train_ds

<DatasetV1Adapter shapes: ({sex: (?,), age: (?,), thalach: (?,)}, (?,)), types: ({sex: tf.string, age: tf.float64, thalach: tf.float64}, tf.float64)>

In [24]:
# create TF numeric feature
tf_numeric_age_feature = tf.feature_column.numeric_column(key='age', default_value=0, dtype=tf.float64)

In [25]:
b_list = [ 0, 18, 25, 40, 55, 65, 80, 100]
#create TF bucket feature from numeric feature
tf_numeric_age_feature
tf_bucket_age_feature = tf.feature_column.bucketized_column(source_column=tf_numeric_age_feature, boundaries= b_list)

In [26]:
#using list b/c small number of unique values
gender_vocab = tf.feature_column.categorical_column_with_vocabulary_list(
      'sex', bp_df['sex'].unique())
gender_one_hot = tf.feature_column.indicator_column(gender_vocab)

In [27]:
# add cross features - use example from TF
crossed_age_gender_feature = tf.feature_column.crossed_column([tf_bucket_age_feature, gender_vocab], hash_bucket_size=1000)
tf_crossed_age_gender_feature = tf.feature_column.indicator_column(crossed_age_gender_feature)

In [28]:
feature_columns = [ tf_crossed_age_gender_feature, tf_bucket_age_feature, gender_one_hot ]
dense_feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [29]:
# Use same architecture as example
def build_model(dense_feature_layer):
  model = tf.keras.Sequential([
    dense_feature_layer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [30]:
model = build_model(dense_feature_layer)

In [31]:
EPOCHS = 1000
early_stop = tf.keras.callbacks.EarlyStopping(monitor='mse', patience=50)     
history = model.fit(train_ds,   callbacks=[early_stop], epochs=EPOCHS,  verbose=1)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 3 steps
Epoch 1/1000


FailedPreconditionError: Table not initialized.
	 [[{{node sequential/dense_features/age_bucketized_X_sex_indicator/hash_table_Lookup/LookupTableFindV2}}]]

In [36]:
loss, mae, mse = model.evaluate(test_ds, verbose=2)
print("MAE:{}\nMSE:{}".format(mae, mse))

Consider rewriting this model with the Functional API.
1/1 - 0s - loss: 338.2635 - mae: 14.7156 - mse: 338.2635
MAE:14.715617179870605
MSE:338.26348876953125


In [37]:
test_labels = test_dataset[PREDICTOR_FIELD].values

In [38]:
test_predictions = model.predict(test_ds).flatten()

Consider rewriting this model with the Functional API.


In [39]:
model_pred_outputs = {
    "pred": test_predictions,
    "actual_value": test_labels,
}
model_output_df = pd.DataFrame(model_pred_outputs)

In [40]:
# Convert Regression Output to binary classification output
model_output_df.tail()

,pred,actual_value
41,126.575775,134.0
42,127.757805,170.0
43,137.408356,125.0
44,126.575775,120.0
45,141.381210,130.0


In [41]:
#convert to binary prediction for Brier score - resting bps above 130 
def convert_to_binary(df, pred_field, actual_field):
    # score is the field name we will use for predictions and is what Aequitas uses
    df['score'] = df[pred_field].apply(lambda x: 1 if x>=130 else 0 )
    # label_value is the field name we will use for the truth value and is what Aequitas uses
    df['label_value'] = df[actual_field].apply(lambda x: 1 if x>=130 else 0)
    return df

In [42]:
binary_df = convert_to_binary(model_output_df, 'pred', 'actual_value')
binary_df.head()

,pred,actual_value,score,label_value
0,134.110626,95.0,1,0
1,141.381210,115.0,1,0
2,140.520813,170.0,1,1
3,126.575775,160.0,0,1
4,140.520813,140.0,1,1


In [43]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score

In [44]:
y_true = binary_df['label_value'].values 
y_pred = binary_df['score'].values

In [45]:
accuracy_score(y_true, y_pred)

0.41304347826086957

In [46]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.28      0.26      0.27        19
           1       0.50      0.52      0.51        27

    accuracy                           0.41        46
   macro avg       0.39      0.39      0.39        46
weighted avg       0.41      0.41      0.41        46



In [47]:
roc_auc_score(y_true, y_pred)

0.3908382066276803